In [1]:
import pandas as pd
import os
import re
import logging
from src.utils import download_images, parse_string
from src.sanity import sanity_check
from src.constants import entity_unit_map
import easyocr

In [2]:
reader = easyocr.Reader(['en'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\Aniruddha\anaconda3\Lib\site-packages\easyocr\detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any i

In [3]:
train_file = 'dataset/train.csv'
train_df = pd.read_csv(train_file)

In [4]:
image_folder = 'downloaded_images'
# Check if the image folder exists, if not, create it
if not os.path.exists(image_folder):
    os.makedirs(image_folder)
    print(f"Directory {image_folder} created.")
else:
    print(f"Directory {image_folder} already exists.")

Directory downloaded_images already exists.


In [5]:
test_file = 'dataset/test.csv'  # Replace with the correct path to test.csv
test_df = pd.read_csv(test_file)

In [6]:
download_images(train_df['image_link'].tolist(), image_folder)


Exception in thread Thread-5 (_handle_workers):
Traceback (most recent call last):
  File "c:\Users\Aniruddha\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\Aniruddha\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Aniruddha\anaconda3\Lib\multiprocessing\pool.py", line 522, in _handle_workers
  0%|          | 0/263859 [00:00<?, ?it/s]    cls._wait_for_updates(current_sentinels, change_notifier)
  File "c:\Users\Aniruddha\anaconda3\Lib\multiprocessing\pool.py", line 502, in _wait_for_updates
    wait(sentinels, timeout=timeout)
  File "c:\Users\Aniruddha\anaconda3\Lib\multiprocessing\connection.py", line 1084, in wait
    ready_handles = _exhaustive_wait(waithandle_to_obj.keys(), timeout)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Aniruddha\anaconda3\Lib\multiprocessing\connection.py", line 1016, in _exhaustive_wait
    res = _winapi.Wa

In [8]:
def extract_entity_value(image_path, entity_name):
    try:
        # Load image and apply OCR using EasyOCR
        result = reader.readtext(image_path, detail=0)
        extracted_text = ' '.join(result)
        
        # Log extracted text for debugging
        logging.debug(f"Extracted Text from {image_path}: {extracted_text}")
        
        # Extract the entity value using simple text search based on the entity name
        entity_value = None
        for entity in entity_unit_map[entity_name]:
            if entity in extracted_text:
                entity_value = entity
                break
        # If entity_value found, extract numerical value
        if entity_value:
            # Extract the first numerical value found in the text
            number = re.findall(r'\d+\.\d+|\d+', extracted_text)
            if number:
                return f"{number[0]} {entity_value}"
            else:
                logging.error(f"No number found for {entity_name} in {image_path}")
        else:
            logging.error(f"No matching entity for {entity_name} in {image_path}")
    finally:
        # If no value found, return an empty string
        return ""

In [9]:
# Predict the entity values for each test image
predictions = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(image_folder, os.path.basename(row['image_link']))
    entity_name = row['entity_name']
    
    if os.path.exists(image_path):
        prediction = extract_entity_value(image_path, entity_name)
    else:
        prediction = ""
    
    predictions.append(prediction)


In [10]:
# Create the output DataFrame
output_df = pd.DataFrame({
    'index': train_df['index'],
    'prediction': predictions
})

In [ ]:
output_df.head()

In [ ]:
output_file = 'predictions.csv'
output_df.to_csv(output_file, index=False)

In [ ]:
# Run the sanity check
sanity_check(test_file, output_file)